<h1>Importing anthropometries, HpFTs, hyperparameters ans setting up constants<h1>

In [1]:
import numpy as np
import pandas as pd
import pickle
from Pipeline import Pipeline
from dsp_utils import *

In [2]:
x_train_df = pd.read_csv('../exported/x_train_df.csv')
""" 
x_train_df.csv contains the training set:
Unnamed 0: the indeces from the original dataframe which has been shuffled and devided into training and test set
Dataset: dataset that example come from
Dataset Indices: subject index relative to the dataset it come from
Anthropometries

x_test_df.csv contains the same informations but for the test set anthrpometries
"""
x_train_df

,Unnamed: 0,Dataset,Dataset Indices,d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,Rotation,Flare,Ear (bin),Ear
0,246,SONICOM,38,1.58825,0.900999,1.66668,2.09915,6.24858,2.96023,0.73,1.01,0.93,0.16,5.337333,29.887155,1.0,Right
1,36,HUTUBS,62,1.71910,0.926500,1.85070,2.23030,6.17460,3.04390,0.38,0.89,1.07,0.27,14.394700,20.443800,0.0,Left
2,114,SONICOM,65,1.36784,0.985845,1.64651,1.43191,5.75449,2.52507,0.60,1.04,1.12,0.32,3.044598,30.042385,0.0,Left
3,24,HUTUBS,44,1.74320,0.944900,1.90920,2.10590,5.94060,3.18550,0.84,1.22,1.32,0.24,13.953300,19.485500,0.0,Left
4,232,SONICOM,22,1.93559,0.803161,1.60971,2.25178,6.94940,3.65465,0.53,1.07,1.13,0.36,15.734473,26.920868,1.0,Right
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,126,SONICOM,79,1.88200,1.165510,1.75890,2.14971,6.86136,3.12623,0.78,1.13,1.24,0.26,9.556536,20.146132,0.0,Left
240,83,SONICOM,26,1.68840,0.927257,1.64323,1.55413,5.90876,2.71771,0.51,1.11,1.09,0.27,0.747495,18.157271,0.0,Left
241,302,SONICOM,172,1.39548,0.882230,1.70717,1.60525,5.51953,2.92887,0.55,0.76,0.88,0.53,2.417278,21.287908,1.0,Right
242,87,SONICOM,31,1.59362,0.976411,1.37537,1.71393,6.27861,3.05929,0.67,1.22,1.22,0.38,6.914592,37.832851,0.0,Left


In [3]:
sr = 44100
dt = 1 / sr
samples = 1882
f = np.linspace(0, sr / 2, int(samples / 2))
t = np.linspace(0, samples * dt, samples)

MODEL_TYPE = 'kn' # example on the kn regression algorithm
LOW_FREQ = f[1]
HIGH_FREQ = f[-2]
TRAIN_INDICES = x_train_df.iloc[:]['Unnamed: 0'].to_numpy()
DATASET_LEN = 306

"""
paths to save/load the models
PATH_KN = 'models/kn/kn.pickle'
PATH_GRNN = 'models/grnn/grnn.pickle'
PATH_RBFNN = 'models/rbfnn/rbfnn.pickle'
PATH_SVR = 'models/svr/svr.pickle'
PATH_FOREST = 'models/forest/forest.pickle'
PATH_FOREST_2 = 'models/forest_2/forest.pickle'
"""

In [4]:
data = np.load('../exported/data.npz')
""" 
data.npz contains:
x_train = training set anthropometries ['d1', 'd2', 'd3', 'd4', 'd5', 'd6', 'd7', 'd8', 'd9', 'd10', 'Rotation', 'Flare', 'Ear']
x_test = test set anthropoemtries
x_train_reduced = training set anthropometries for the second random forest ['d1', 'd2', 'd3', 'd4', 'd5', 'd6', 'd7','Rotation']
x_test_reduced = test set anthropometries for the second random forest
y_train = training set HpTF (magnitude spectrum in dB)
y_test = test set HpTF (magnitude spectrum in dB)
"""
x_train = data['x_train']
x = data['x_test'] # unknown anthropometries to make predictions on
y_train = data['y_train']
y = data['y_test'] # ground truth of unknonwn input anthropometries

In [5]:
# loading hyperparameters
with open('../exported/hyperparams.pkl', 'rb') as file:
    hyperparams = pickle.load(file)
"""
hyperparams is a dictionary with list of hyperparameters for the learning algorithms explored
keys are: 'kn', 'svr', 'grnn', 'forest', 'forest_2', 'rbfnn'
"""

"\nhyperparams is a dictionary with list of hyperparameters for the learning algorithms explored\nkeys are: 'kn', 'svr', 'grnn', 'forest', 'forest_2', 'rbfnn'\n"

In [6]:
hyperparams

{'kn': (20, 'cosine'),
 'svr': ['rbf'],
 'grnn': [0.25],
 'forest': (7, True),
 'forest_2': (5, True),
 'rbfnn': [306]}

In [7]:
# loading processing hyperparameters (to choose the pre-processing for anthropometries and target HpTFs)
with open('../exported/processing.pkl', 'rb') as file:
    processing = pickle.load(file)
"""
dictionary keys are: 'kn', 'svr', 'grnn', 'forest', 'forest_2', 'rbfnn'
values are tuples (pre-processing, post-processing)
"""

"\ndictionary keys are: 'kn', 'svr', 'grnn', 'forest', 'forest_2', 'rbfnn'\nvalues are tuples (pre-processing, post-processing)\n"

<h1>Istantiating, building and training the model<h1>

In [8]:
# istantiating, building and train the model

model = Pipeline( MODEL_TYPE )
""" 
Pipeline: istantiating the model
INPUT:
- model_type: either one of 'kn', 'grnn', 'rbfnn', 'forest', 'svr'
"""

model.computeInputPreprocessingParams(processing['kn'][0], x_train, DATASET_LEN, TRAIN_INDICES, y_train)
"""
computeInputPreprocessingParams: compute internal parameters for the ML algorithm given training set info
INPUT:
- preprocessing_type: type of input pre-processing, either 'none', 'pca, 'selection'
- x: training set used to compute normalization parameters

if preprocessing_type == 'selection'
- original dataset length
- training set indices
- training set HpTFs
"""

x_train_preprocessed = model.inputPreprocessing( x_train )
""" 
inputPreprocessing: apply pre-processing on the training set
INPUT:
- x: training set
"""

model.computeOutputPreprocessingParams(processing['kn'][1], y_train)
"""
computeOutputPreprocessingParams: compute internal parameters for the ML algorithm given the pre-processing technique to use on the hptfs
"""

y_train_preprocessed = model.outputPreprocessing( y_train )
""" 
"""

model.modelBuild( hyperparams['kn'] )
"""
modelBuild: set algorithm hyperparameters
"""

model.modelTrain( x_train_preprocessed, y_train_preprocessed )
""" 
modelTrain: train the model on the training set
INPUT:
- x_train_preprocessed: training set inputs preprocessed
- y_train_preprocessed: training set targets preprocessed
"""

' \nmodelTrain: train the model on the training set\nINPUT:\n- x_train_preprocessed: training set inputs preprocessed\n- y_train_preprocessed: training set targets preprocessed\n'

<h1>Saving and loading the model<h1>

In [9]:
# save the model
#saveModel(PATH, model, MODEL_TYPE)

# if the model already trained and built then load model:
#model = loadModel(PATH, MODEL_TYPE)

<h1>Make predictions and process predicted HpTFs<h1>

In [10]:
# predictions on test set
x_preprocessed = model.inputPreprocessing( x )
predictions = model.modelPredict( x_preprocessed )
predictions = model.outputConvert( predictions ) # predicted HpTFs (magnitude spectrum in dB)

predictions_minimum = minimumPhaseSpectrum(ct.db2mag(predictions), 's') # minimum spectrum predicted HpTFs
"""
minimumPhaseSpectrum: compute minimum phase transfer function
INPUT:
- magnitude_spectrum: magnitude spectrum of an transfer function (not in dB)
- side: either 's' or 'd', if 's', then magnitude_spectrum is a single sided spectrum, if 'd' then magnitude_spectrum is a double sided spectrum

always return the single sided frequency spectrum of the given magnitude spectrum
"""

predictions_inverse, _, _ = inverseLMSRegularized( spec2mag(predictions_minimum, 's'), LOW_FREQ, HIGH_FREQ, f, sr )
"""
inverseLMSRegularized: inverse the given transfer function
INPUT
- one_sided_spectrum: one sided magnitude spectrum
- low_freq: low frequency limit for inversion
- high_freq: high frequency limit or inversion
- f: frequency vector
- sr: sample rate

RETURN
- inverse transfer function frequency spectrum
- regularization effort alpha vector
- regularization effor sigma vector
"""

'\ninverseLMSRegularized: inverse the given transfer function\nINPUT\n- one_sided_spectrum: one sided magnitude spectrum\n- low_freq: low frequency limit for inversion\n- high_freq: high frequency limit or inversion\n- f: frequency vector\n- sr: sample rate\n\nRETURN\n- inverse transfer function frequency spectrum\n- regularization effort alpha vector\n- regularization effor sigma vector\n'

<h1>RBFNN model building<h1>

In [ ]:
if processing['rbfnn'][1] == 'pca':
    loss = 'mse'
else:
    loss = 'custom'

MODEL_TYPE = 'rbfnn'

model = Pipeline( MODEL_TYPE )
model.computeInputPreprocessingParams(processing['rbfnn'][0], x_train, DATASET_LEN, TRAIN_INDICES, y_train) 
x_train_preprocessed = model.inputPreprocessing( x_train )
model.computeOutputPreprocessingParams(processing['rbfnn'][1], y_train)
y_train_preprocessed = model.outputPreprocessing( y_train )
model.modelBuild( hyperparams['rbfnn'], x_train_preprocessed, y_train_preprocessed.shape[1], loss )
""" 
INPUT
- rbfnn algorithm hyperparameters
- preprocessed train set antrhopometries (is used to initialize the center of the RBF in the neurons)
- preprocessed train set targets shape
- losss function to use
"""
model.modelTrain( x_train_preprocessed, y_train_preprocessed )

# load rbfnn, model and weights
#model_rbfnn = loadModel(PATH_RBFNN, model_type)
#model_rbfnn.model.load_weights('models/rbfnn/checkpoint.weights.h5')